In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [8]:
print(tf.__version__)

1.14.0


In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 54000 samples, validate on 6000 samples
Epoch 1/4
54000/54000 [==============================] - 11s 195us/sample - loss: 0.2849 - acc: 0.9209 - val_loss: 0.1157 - val_acc: 0.9698
Epoch 2/4
54000/54000 [==============================] - 2s 44us/sample - loss: 0.1091 - acc: 0.9686 - val_loss: 0.0852 - val_acc: 0.9753
Epoch 3/4
54000/54000 [==============================] - 2s 43us/sample - loss: 0.0816 - acc: 0.9761 - val_loss: 0.0678 - val_acc: 0.9812
Epoch 4/4
54000/54000 [==============================] - 2s 44us/sample - loss: 0.0688 - acc: 0.9793 - val_loss: 0.0609 - val_acc: 0.9842


In [3]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9773
Saved baseline model to: /tmp/tmp8lkzu9gm.h5


In [14]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

input_shape=[20]
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

#model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(20, input_shape=input_shape), **pruning_params),
  
])

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_dense_3  (None, 20)                822       
Total params: 822
Trainable params: 420
Non-trainable params: 402
_________________________________________________________________


In [5]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 2s 42us/sample - loss: 0.0792 - acc: 0.9773 - val_loss: 0.1082 - val_acc: 0.9765
Epoch 2/2
54000/54000 [==============================] - 2s 28us/sample - loss: 0.0992 - acc: 0.9737 - val_loss: 0.0798 - val_acc: 0.9793


In [6]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9773
Pruned test accuracy: 0.972


In [7]:
%tensorboard --logdir={logdir}